In [2]:
import os
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cross_validation import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [3]:
cwd = os.getcwd()
train_file = os.path.join(cwd, "traffic-signs-data/train.p")
test_file = os.path.join(cwd, "traffic-signs-data/test.p")

img_h, img_w = 32, 32

In [4]:
with open(train_file, 'rb') as f:
    train = pickle.load(f)
with open(test_file, 'rb') as f:
    test = pickle.load(f)

In [5]:
train.keys(), test.keys()

(dict_keys(['sizes', 'coords', 'features', 'labels']),
 dict_keys(['sizes', 'coords', 'features', 'labels']))

In [6]:
train_images, train_labels = train['features'], train['labels']
test_images, test_labels = test['features'], test['labels']

In [7]:
def preprocess(images, equalize_hist=True):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    preprocessed_images = np.expand_dims(np.zeros_like(images)[:, :, :, 0], 3)
    for i in range(len(images)):
        img = cv2.cvtColor(images[i], cv2.COLOR_RGB2YUV)[:, :, 0]
        if equalize_hist:
#             img = cv2.equalizeHist(img)
            img = clahe.apply(img)
        preprocessed_images[i] = np.expand_dims(img, 2)
    preprocessed_images = np.float32(preprocessed_images)
#     images -= np.mean(images, axis=0)
#     images /= (np.std(images, axis=0) + np.finfo('float32').eps)
    preprocessed_images = (preprocessed_images/127.5) - 1
    return preprocessed_images

In [8]:
train_preprocessed_images = preprocess(train_images)
test_preprocessed_images = preprocess(test_images)

In [10]:
train_preprocessed_images.shape, test_preprocessed_images.shape

((39209, 32, 32, 1), (12630, 32, 32, 1))

In [11]:
def create_model(num_classes, input_shape=(32, 32, 1), lr=0.0001):
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu', input_shape=input_shape))
    model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(units=1024, activation='relu'))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=43, activation='softmax'))
    print(model.summary())

    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [12]:
input_shape = (img_h, img_w, 1)
model = create_model(num_classes=43, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        102464    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 128)         147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
__________

In [14]:
val_size = 0.15

train_labels_one_hot = keras.utils.to_categorical(train_labels, num_classes=43)

x_train, x_val, y_train, y_val = train_test_split(train_preprocessed_images, train_labels_one_hot, test_size=val_size)
x_train.shape, x_val.shape

((33327, 32, 32, 1), (5882, 32, 32, 1))

In [15]:
train_generator = ImageDataGenerator(rotation_range=15., zoom_range=0.2, width_shift_range=0.1, height_shift_range=0.1)
val_generator = ImageDataGenerator()

train_generator.fit(x_train)
val_generator.fit(x_val)

In [16]:
batch_size = 32
epochs = 20
model.fit_generator(train_generator.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=len(x_train)/ batch_size, epochs=epochs, \
                    validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), \
                    validation_steps=len(x_val)/batch_size)

Epoch 1/20
1042/1041 [==============================] - 23s 22ms/step - loss: 2.3987 - acc: 0.3290 - val_loss: 0.8969 - val_acc: 0.7101
Epoch 2/20
1042/1041 [==============================] - 18s 17ms/step - loss: 1.1262 - acc: 0.6519 - val_loss: 0.2952 - val_acc: 0.9109
Epoch 3/20
1042/1041 [==============================] - 21s 20ms/step - loss: 0.7122 - acc: 0.7724 - val_loss: 0.1569 - val_acc: 0.9538
Epoch 4/20
1042/1041 [==============================] - 23s 22ms/step - loss: 0.5075 - acc: 0.8353 - val_loss: 0.1051 - val_acc: 0.9691
Epoch 5/20
1042/1041 [==============================] - 26s 25ms/step - loss: 0.3939 - acc: 0.8709 - val_loss: 0.0727 - val_acc: 0.9777
Epoch 6/20
1042/1041 [==============================] - 33s 32ms/step - loss: 0.3270 - acc: 0.8956 - val_loss: 0.0589 - val_acc: 0.9821
Epoch 7/20
1042/1041 [==============================] - 31s 30ms/step - loss: 0.2722 - acc: 0.9116 - val_loss: 0.0432 - val_acc: 0.9881
Epoch 8/20
1042/1041 [==========================

In [34]:
predicted = model.predict(test_preprocessed_images)

In [35]:
predicted = np.argmax(predicted, axis=1)
predicted.shape

(12630,)

In [36]:
import sklearn
accuracy_score = sklearn.metrics.accuracy_score(y_true=test_labels, y_pred=predicted)

In [37]:
accuracy_score

0.9851148060174189